In [ ]:
# Import necessary libraries
# JupyterDash is used to run Dash inside Jupyter Notebook
from jupyter_dash import JupyterDash

# Dash components for UI elements
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64  # Used for encoding images

# OS and data-handling libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import custom CRUD module for database interactions
from aac_crud import AnimalShelter

###########################################
# Environment Variables and Database Setup
###########################################

# Using environment variables for **security** instead of hardcoded credentials.
# If not set, default values are used.
username = os.getenv("AAC_USERNAME", "aacuser")  
password = os.getenv("AAC_PASSWORD", "SNHU1234")  

# Database connection parameters
HOST = 'nv-desktop-services.apporto.com'
PORT = 32814
DB = 'AAC'
COL = 'animals'

# Establish connection to MongoDB using AnimalShelter CRUD module
db = AnimalShelter(username, password, HOST, PORT, DB, COL)

###########################################
# Helper Functions
###########################################

def clean_dataframe(df):
    """
    Removes unnecessary MongoDB `_id` column.
    Some MongoDB versions include an ObjectID column that can cause crashes when rendering.
    This function ensures that the dataframe is cleaned before being used in the dashboard.
    """
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df

# Fetch initial dataset of all dogs in the database
df = clean_dataframe(pd.DataFrame.from_records(db.read({'animal_type': 'Dog'})))

###########################################
# Dash Web Application Setup
###########################################

# Initialize JupyterDash instance
app = JupyterDash(__name__)

# **Handling Images Securely**
# Instead of assuming an image file exists, we check first to avoid errors
image_filename = 'Grazioso_Salvare_Logo.png'
encoded_image = None  # Default: No image until verified
if os.path.exists(image_filename):  # Check if file exists before reading
    with open(image_filename, 'rb') as img_file:
        encoded_image = base64.b64encode(img_file.read())  # Encode image for Dash display

###########################################
# Dashboard Layout
###########################################

app.layout = html.Div([
    # Header Section
    html.Div(className='row', style={'display': 'flex', 'margin': '10px'},
        children=[
            html.Img(src=f'data:image/png;base64,{encoded_image.decode()}' if encoded_image else '',
                     style={'height': '5%', 'width': '5%'}),
            html.Center(html.B(html.H1('AAC Grazioso Salvare Dashboard - Benjamin Sturgeon')))
        ]),

    html.Hr(),  # Horizontal divider for better UI structure
    
    # **Filtering options** to dynamically update table based on rescue type
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset'  # Default value set to 'Reset'
    ),

    html.Hr(),

    # **Interactive Data Table** displaying filtered animal rescue data
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False, 
        filter_action="native",  
        sort_action="native",  
        sort_mode="multi",  
        column_selectable="single", 
        row_selectable="single",
        row_deletable=False,  
        page_action="native", 
        page_current=0,  
        page_size=10  
    ),

    html.Br(),
    html.Hr(),

    # **Row Layout for Graph and Map Sections**
    html.Div(className='row', style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),  # Placeholder for charts
            html.Div(id='map-id', className='col s12 m6')  # Placeholder for map
        ])
])

###########################################
# Callback Functions
###########################################

@app.callback(Output('datatable-id', 'data'), [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    """
    Updates the data table based on selected rescue type.
    Uses MongoDB queries to filter animals dynamically.
    """
    filters = {
        'Water': {
            "$and": [
                {"$or": [{'breed': {"$regex": 'Labrador Retriever Mix'}},
                         {'breed': {"$regex": 'Chesapeake'}},
                         {'breed': {"$regex": 'Newfoundland'}}]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {"$gt": 26, "$lt": 156}}
            ]
        },
        'Mountain': {
            "$and": [
                {"$or": [{'breed': {"$regex": 'German Shepherd'}},
                         {'breed': {"$regex": 'Alaskan Malamute'}},
                         {'breed': {"$regex": 'Old English Sheepdog'}},
                         {'breed': {"$regex": 'Siberian Husky'}},
                         {'breed': {"$regex": 'Rottweiler'}}]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gt": 26, "$lt": 156}}
            ]
        },
        'Disaster': {
            "$and": [
                {"$or": [{'breed': {"$regex": 'Doberman Pinscher'}},
                         {'breed': {"$regex": 'German Shepherd'}},
                         {'breed': {"$regex": 'Golden Retriever'}},
                         {'breed': {"$regex": 'Bloodhound'}},
                         {'breed': {"$regex": 'Rottweiler'}}]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {"$gt": 20, "$lt": 300}}
            ]
        }
    }
    
    query = filters.get(filter_type, {'animal_type': 'Dog'})  # Default case for unrecognized input
    df = clean_dataframe(pd.DataFrame.from_records(db.read(query)))
    return df.to_dict('records')

@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    """
    Displays a Pie Chart representing breed distribution in the dataset.
    Dynamically updates when filtered data changes.
    """
    if viewData is None:
        return []
    df = pd.DataFrame.from_records(viewData)
    return [dcc.Graph(figure=px.pie(df, names='breed', title='Preferred Animals'))]

@app.callback(Output('datatable-id', 'style_data_conditional'), [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    """
    Highlights the column selected in the data table for easier visibility.
    """
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

@app.callback(Output('map-id', "children"), [Input('datatable-id', "derived_virtual_data"),
                                             Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    """
    Displays location-based map of selected animal entry.
    Users select an entry from the table, and it updates the corresponding pin.
    """
    if viewData is None or index is None:
        return []
    
    df = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[df.iloc[row, 13], df.iloc[row, 14]], children=[
            dl.Tooltip(df.iloc[row, 4]),
            dl.Popup([html.H1("Animal Name"), html.P(df.iloc[row, 9])])
        ])
    ])]

# Start the Dash application
app.run_server(debug=True)

Dash app running on http://127.0.0.1:14685/
